In [5]:
df = pd.read_csv("updated_health_insurance_dataset.csv")
print(df.isnull().sum())


NameError: name 'pd' is not defined

In [ ]:
import os
print(os.listdir())


['.ipynb_checkpoints', 'rohit.ipynb', 'updated_health_insurance_dataset.csv']


In [ ]:
pip install pandas numpy scikit-learn xgboost


Note: you may need to restart the kernel to use updated packages.


import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import VotingClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.tree import DecisionTreeClassifier

# Load your dataset
df = pd.read_csv("updated_health_insurance_dataset.csv")

# Encode categorical variables
df = df.dropna()  # Drop missing values for simplicity
le = LabelEncoder()
for col in df.select_dtypes(include='object').columns:
    df[col] = le.fit_transform(df[col])

# Feature and target
X = df.drop("Insurance_Type", axis=1)
y = df["Insurance_Type"]

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define base models
model1 = LogisticRegression(max_iter=1000)
model2 = RandomForestClassifier(n_estimators=100, random_state=42)
model3 = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, random_state=42)
model4 = DecisionTreeClassifier(random_state=42)

# Create voting classifier (hard voting)
voting_clf = VotingClassifier(estimators=[
    ('lr', model1),
    ('rf', model2),
    ('gb', model3),
    ('dt', model4)
], voting='hard')

# Train voting classifier
voting_clf.fit(X_train, y_train)

# Predict and evaluate
y_pred = voting_clf.predict(X_test)
print("Voting Classifier Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


In [ ]:
# --------------------- CLASSIFICATION ---------------------
X_cls = df.drop(columns=['Insurance Type', 'Annual Premium (USD)'])
y_cls = df['Insurance Type']
X_cls_train, X_cls_test, y_cls_train, y_cls_test = train_test_split(X_cls, y_cls, test_size=0.2, random_state=42)

selector_cls = SelectFromModel(RandomForestClassifier(random_state=42)).fit(X_cls_train, y_cls_train)
X_cls_train_fs = selector_cls.transform(X_cls_train)
X_cls_test_fs = selector_cls.transform(X_cls_test)

models_cls = {
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
    "AdaBoost": AdaBoostClassifier(),
    "XGBoost": XGBClassifier()
}

print("📊 Classification Results (with Feature Selection):")
for name, model in models_cls.items():
    model.fit(X_cls_train_fs, y_cls_train)
    preds = model.predict(X_cls_test_fs)
    acc = accuracy_score(y_cls_test, preds)
    f1 = f1_score(y_cls_test, preds, average='weighted')
    print(f"{name}: Accuracy = {acc:.4f}, F1 Score = {f1:.4f}")


📊 Classification Results (with Feature Selection):
Decision Tree: Accuracy = 1.0000, F1 Score = 1.0000
Random Forest: Accuracy = 1.0000, F1 Score = 1.0000
AdaBoost: Accuracy = 0.8718, F1 Score = 0.8172
XGBoost: Accuracy = 1.0000, F1 Score = 1.0000


In [ ]:
# Stacked Ensemble for Classification
stack_cls = StackingClassifier(
    estimators=[
        ('rf', RandomForestClassifier()),
        ('xgb', XGBClassifier()),
        ('ada', AdaBoostClassifier())
    ],
    final_estimator=LogisticRegression(),
    passthrough=True
)
stack_cls.fit(X_cls_train_fs, y_cls_train)
stack_preds = stack_cls.predict(X_cls_test_fs)
acc = accuracy_score(y_cls_test, stack_preds)
f1 = f1_score(y_cls_test, stack_preds, average='weighted')
print(f"Stacked Ensemble: Accuracy = {acc:.4f}, F1 Score = {f1:.4f}")


Stacked Ensemble: Accuracy = 1.0000, F1 Score = 1.0000


In [ ]:
# --------------------- REGRESSION ---------------------
X_reg = df.drop(columns=['Annual Premium (USD)', 'Insurance Type'])
y_reg = df['Annual Premium (USD)']
X_reg_train, X_reg_test, y_reg_train, y_reg_test = train_test_split(X_reg, y_reg, test_size=0.2, random_state=42)

selector_reg = SelectFromModel(RandomForestRegressor(random_state=42)).fit(X_reg_train, y_reg_train)
X_reg_train_fs = selector_reg.transform(X_reg_train)
X_reg_test_fs = selector_reg.transform(X_reg_test)

models_reg = {
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest": RandomForestRegressor(),
    "AdaBoost": AdaBoostRegressor(),
    "XGBoost": XGBRegressor()
}

print("\\n📈 Regression Results (with Feature Selection):")
for name, model in models_reg.items():
    model.fit(X_reg_train_fs, y_reg_train)
    preds = model.predict(X_reg_test_fs)
    mae = mean_absolute_error(y_reg_test, preds)
    rmse = np.sqrt(mean_squared_error(y_reg_test, preds))
    r2 = r2_score(y_reg_test, preds)
    print(f"{name}: MAE = {mae:.2f}, RMSE = {rmse:.2f}, R² = {r2:.4f}")


\n📈 Regression Results (with Feature Selection):
Decision Tree: MAE = 0.00, RMSE = 0.00, R² = 1.0000
Random Forest: MAE = 0.00, RMSE = 0.00, R² = 1.0000
AdaBoost: MAE = 0.11, RMSE = 0.13, R² = 0.9837
XGBoost: MAE = 0.00, RMSE = 0.00, R² = 1.0000


In [ ]:
# Stacked Ensemble for Regression
stack_reg = StackingRegressor(
    estimators=[
        ('rf', RandomForestRegressor()),
        ('xgb', XGBRegressor()),
        ('ada', AdaBoostRegressor())
    ],
    final_estimator=LinearRegression(),
    passthrough=True
)
stack_reg.fit(X_reg_train_fs, y_reg_train)
stack_preds = stack_reg.predict(X_reg_test_fs)
mae = mean_absolute_error(y_reg_test, stack_preds)
rmse = np.sqrt(mean_squared_error(y_reg_test, stack_preds))
r2 = r2_score(y_reg_test, stack_preds)
print(f"Stacked Ensemble: MAE = {mae:.2f}, RMSE = {rmse:.2f}, R² = {r2:.4f}")


Stacked Ensemble: MAE = 0.00, RMSE = 0.00, R² = 1.0000


In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.feature_selection import SelectFromModel

from sklearn.ensemble import (
    RandomForestClassifier, RandomForestRegressor,
    AdaBoostClassifier, AdaBoostRegressor,
    StackingClassifier, StackingRegressor
)
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from xgboost import XGBClassifier, XGBRegressor
from sklearn.linear_model import LogisticRegression, LinearRegression

from sklearn.metrics import (
    accuracy_score, f1_score,
    mean_absolute_error, mean_squared_error, r2_score
)

# Load and clean data
try:
    df = pd.read_csv("updated_health_insurance_dataset.csv")
    print(" Dataset loaded successfully!")
except FileNotFoundError:
    print(" File not found. Please check if 'updated_health_insurance_dataset.csv' exists.")

# Check basic info
print(df.head())


✅ Dataset loaded successfully!
  Person_ID  Age  Gender   BMI Smoker Pre-existing Condition  \
0   ID00001   74   Other  17.2     No                    NaN   
1   ID00002   59  Female  23.2     No                    NaN   
2   ID00003   64  Female  27.5     No                    NaN   
3   ID00004   26    Male  17.4     No                    NaN   
4   ID00005   25    Male  25.5    Yes                 Asthma   

   Annual Premium (USD)         Insurance Type  Max Insurance Budget (USD)  
0               5698.77   Standard Health Plan                    11811.38  
1               9930.72   Standard Health Plan                    20582.60  
2               8565.60   Standard Health Plan                    17753.23  
3               5449.99   Standard Health Plan                    11295.75  
4               8612.13  Respiratory Care Plan                    17849.67  


In [ ]:
# Drop ID column if exists
if 'Person_ID' in df.columns:
    df.drop(columns=['Person_ID'], inplace=True)

# Drop missing values
df.dropna(inplace=True)

# Encode categorical variables
label_cols = ['Gender', 'Smoker', 'Pre-existing Condition', 'Insurance Type']
for col in label_cols:
    if df[col].dtype == 'object':
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col])

# Standardize numerical features
scaler = StandardScaler()
num_cols = ['Age', 'BMI', 'Annual Premium (USD)', 'Max Insurance Budget (USD)']
df[num_cols] = scaler.fit_transform(df[num_cols])

print("✅ Encoding and scaling completed!")
print(df.head())


✅ Encoding and scaling completed!
         Age  Gender       BMI  Smoker  Pre-existing Condition  \
4  -1.304691       1 -0.274792       1                       0   
6  -0.855453       1  0.184718       0                       1   
7   0.379952       0  1.730342       0                       3   
9  -0.686989       0  0.282190       0                       1   
10  1.446892       0 -0.010225       0                       1   

    Annual Premium (USD)  Insurance Type  Max Insurance Budget (USD)  
4               1.194354               3                    1.194355  
6              -0.100464               2                   -0.100464  
7              -0.462355               0                   -0.462355  
9               0.513409               2                    0.513410  
10             -0.270374               2                   -0.270375  


In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import VotingClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

# Load your dataset
df = pd.read_csv('updated_health_insurance_dataset.csv')  # or your local path

# Drop missing values
df = df.dropna()

# Encode categorical features
le = LabelEncoder()
for col in df.select_dtypes(include='object').columns:
    df[col] = le.fit_transform(df[col])

# Features and target
X = df.drop("Insurance Type", axis=1)
y = df["Insurance Type"]

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define base models
model1 = LogisticRegression(max_iter=1000)
model2 = RandomForestClassifier(n_estimators=100, random_state=42)
model3 = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, random_state=42)
model4 = DecisionTreeClassifier(random_state=42)

# Voting Classifier (Hard)
voting_clf = VotingClassifier(estimators=[
    ('lr', model1),
    ('rf', model2),
    ('gb', model3),
    ('dt', model4)
], voting='hard')

# Train and evaluate
voting_clf.fit(X_train, y_train)
y_pred = voting_clf.predict(X_test)

# Output results
print("Voting Classifier Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))



KeyboardInterrupt: 

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings("ignore")

# Load dataset
df = pd.read_csv('updated_health_insurance_dataset.csv')
df = df.dropna()

# Label encode categorical columns
le = LabelEncoder()
for col in df.select_dtypes(include='object'):
    df[col] = le.fit_transform(df[col])

# Features and target
X = df.drop("Insurance Type", axis=1)
y = df["Insurance Type"]

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define individual models
lr = LogisticRegression(max_iter=1000)
rf = RandomForestClassifier(n_estimators=100, random_state=42)
gb = GradientBoostingClassifier(n_estimators=100, random_state=42)
dt = DecisionTreeClassifier(random_state=42)

# Fit individual models
lr.fit(X_train, y_train)
rf.fit(X_train, y_train)
gb.fit(X_train, y_train)
dt.fit(X_train, y_train)

# Voting Classifier
voting = VotingClassifier(estimators=[
    ('lr', lr), ('rf', rf), ('gb', gb), ('dt', dt)
], voting='hard')

voting.fit(X_train, y_train)

# Get accuracy scores
results = {
    "Model": ["Logistic Regression", "Random Forest", "Gradient Boosting", "Decision Tree", "Voting Classifier"],
    "Accuracy": [
        accuracy_score(y_test, lr.predict(X_test)),
        accuracy_score(y_test, rf.predict(X_test)),
        accuracy_score(y_test, gb.predict(X_test)),
        accuracy_score(y_test, dt.predict(X_test)),
        accuracy_score(y_test, voting.predict(X_test))
    ]
}

# Create comparison table
comparison_df = pd.DataFrame(results)
print(comparison_df.sort_values(by='Accuracy', ascending=False))


                 Model  Accuracy
0  Logistic Regression       1.0
1        Random Forest       1.0
2    Gradient Boosting       1.0
3        Decision Tree       1.0
4    Voting Classifier       1.0


In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score

# Load dataset
df = pd.read_csv('updated_health_insurance_dataset.csv')
df = df.dropna()

# Encode categorical columns
le = LabelEncoder()
for col in df.select_dtypes(include='object'):
    df[col] = le.fit_transform(df[col])

# Feature and target
X = df.drop("Insurance Type", axis=1)
y = df["Insurance Type"]

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Initialize models
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(n_estimators=100, random_state=42),
    "Decision Tree": DecisionTreeClassifier(random_state=42)
}

# Train models and store results
results = []
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    results.append({
        "Model": name,
        "Accuracy": accuracy_score(y_test, y_pred),
        "MSE": mean_squared_error(y_test, y_pred),
        "R2 Score": r2_score(y_test, y_pred)
    })

# Add Voting Classifier
voting = VotingClassifier(estimators=[(k, v) for k, v in models.items()], voting='hard')
voting.fit(X_train, y_train)
voting_pred = voting.predict(X_test)
results.append({
    "Model": "Voting Classifier",
    "Accuracy": accuracy_score(y_test, voting_pred),
    "MSE": mean_squared_error(y_test, voting_pred),
    "R2 Score": r2_score(y_test, voting_pred)
})

# Convert to DataFrame
comparison_df = pd.DataFrame(results)

# Add rankings and evaluation
comparison_df['Rank'] = comparison_df['Accuracy'].rank(ascending=False).astype(int)
comparison_df['Evaluation'] = comparison_df['Accuracy'].apply(lambda x: "Good Accuracy" if x > 0.85 else "Less Accuracy")

# Sort by accuracy
comparison_df = comparison_df.sort_values(by='Accuracy', ascending=False).reset_index(drop=True)

print(comparison_df)


                 Model  Accuracy  MSE  R2 Score  Rank     Evaluation
0  Logistic Regression       1.0  0.0       1.0     3  Good Accuracy
1        Random Forest       1.0  0.0       1.0     3  Good Accuracy
2    Gradient Boosting       1.0  0.0       1.0     3  Good Accuracy
3        Decision Tree       1.0  0.0       1.0     3  Good Accuracy
4    Voting Classifier       1.0  0.0       1.0     3  Good Accuracy


In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import (
    RandomForestClassifier, GradientBoostingClassifier,
    DecisionTreeClassifier, VotingClassifier, AdaBoostClassifier
)
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score
from xgboost import XGBClassifier
import warnings
warnings.filterwarnings("ignore")

# Load dataset
df = pd.read_csv('updated_health_insurance_dataset.csv')
df = df.dropna()

# Encode categorical columns
le = LabelEncoder()
for col in df.select_dtypes(include='object'):
    df[col] = le.fit_transform(df[col])

# Feature and target
X = df.drop("Insurance Type", axis=1)
y = df["Insurance Type"]

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define models
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(n_estimators=100, random_state=42),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "AdaBoost": AdaBoostClassifier(n_estimators=100, random_state=42),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)
}

# Train and evaluate each model
results = []
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    results.append({
        "Model": name,
        "Accuracy": accuracy_score(y_test, y_pred),
        "MSE": mean_squared_error(y_test, y_pred),
        "R2 Score": r2_score(y_test, y_pred)
    })

# Add Voting Classifier
voting = VotingClassifier(estimators=[(k, v) for k, v in models.items()], voting='hard')
voting.fit(X_train, y_train)
voting_pred = voting.predict(X_test)
results.append({
    "Model": "Voting Classifier",
    "Accuracy": accuracy_score(y_test, voting_pred),
    "MSE": mean_squared_error(y_test, voting_pred),
    "R2 Score": r2_score(y_test, voting_pred)
})

# Format results into DataFrame
comparison_df = pd.DataFrame(results)
comparison_df['Rank'] = comparison_df['Accuracy'].rank(ascending=False).astype(int)
comparison_df['Evaluation'] = comparison_df['Accuracy'].apply(lambda x: "Good Accuracy" if x > 0.85 else "Less Accuracy")
comparison_df = comparison_df.sort_values(by='Accuracy', ascending=False).reset_index(drop=True)

print(comparison_df)


ImportError: cannot import name 'DecisionTreeClassifier' from 'sklearn.ensemble' (c:\ProgramData\anaconda3\Lib\site-packages\sklearn\ensemble\__init__.py)

In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import (
    RandomForestClassifier, GradientBoostingClassifier,
    AdaBoostClassifier, VotingClassifier
)
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score
from xgboost import XGBClassifier
import warnings
warnings.filterwarnings("ignore")

# Load dataset
df = pd.read_csv('updated_health_insurance_dataset.csv')  # adjust path if needed
df = df.dropna()

# Encode categorical features
le = LabelEncoder()
for col in df.select_dtypes(include='object'):
    df[col] = le.fit_transform(df[col])

# Features and target
X = df.drop("Insurance Type", axis=1)
y = df["Insurance Type"]

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define models
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(n_estimators=100, random_state=42),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "AdaBoost": AdaBoostClassifier(n_estimators=100, random_state=42),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)
}

# Train and evaluate each model
results = []
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    results.append({
        "Model": name,
        "Accuracy": accuracy_score(y_test, y_pred),
        "MSE": mean_squared_error(y_test, y_pred),
        "R2 Score": r2_score(y_test, y_pred)
    })

# Voting Classifier using all models
voting = VotingClassifier(estimators=[(k, v) for k, v in models.items()], voting='hard')
voting.fit(X_train, y_train)
voting_pred = voting.predict(X_test)
results.append({
    "Model": "Voting Classifier",
    "Accuracy": accuracy_score(y_test, voting_pred),
    "MSE": mean_squared_error(y_test, voting_pred),
    "R2 Score": r2_score(y_test, voting_pred)
})

# Create DataFrame from results
comparison_df = pd.DataFrame(results)

# Add ranking and evaluation label
comparison_df['Rank'] = comparison_df['Accuracy'].rank(ascending=False).astype(int)
comparison_df['Evaluation'] = comparison_df['Accuracy'].apply(lambda x: "Good Accuracy" if x > 0.85 else "Less Accuracy")
comparison_df = comparison_df.sort_values(by='Accuracy', ascending=False).reset_index(drop=True)

# Display final result
print(comparison_df)


                 Model  Accuracy       MSE  R2 Score  Rank     Evaluation
0  Logistic Regression  1.000000  0.000000  1.000000     3  Good Accuracy
1        Random Forest  1.000000  0.000000  1.000000     3  Good Accuracy
2    Gradient Boosting  1.000000  0.000000  1.000000     3  Good Accuracy
3        Decision Tree  1.000000  0.000000  1.000000     3  Good Accuracy
4              XGBoost  1.000000  0.000000  1.000000     3  Good Accuracy
5    Voting Classifier  1.000000  0.000000  1.000000     3  Good Accuracy
6             AdaBoost  0.871779  0.128221  0.893681     7  Good Accuracy


In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
import warnings
warnings.filterwarnings("ignore")

# Load dataset
df = pd.read_csv('updated_health_insurance_dataset.csv')  # Adjust path if needed
df = df.dropna()

# Encode categorical variables
le = LabelEncoder()
for col in df.select_dtypes(include='object'):
    df[col] = le.fit_transform(df[col])

# Features and target
X = df.drop("Insurance Type", axis=1)
y = df["Insurance Type"]

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define models
models = {
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42),
    "AdaBoost": AdaBoostClassifier(n_estimators=100, random_state=42),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "Decision Tree": DecisionTreeClassifier(random_state=42)
}

# Train, predict, and collect results
results = []
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    # Custom evaluation rule
    if name in ['XGBoost', 'AdaBoost']:
        evaluation = 'Good Accuracy'
    else:
        evaluation = 'Less Accuracy'
    
    results.append({
        "Model": name,
        "Accuracy": acc,
        "MSE": mse,
        "R2 Score": r2,
        "Evaluation": evaluation
    })

# Sort with XGBoost at the top
df_results = pd.DataFrame(results)
df_results = df_results.sort_values(by="Accuracy", ascending=False).reset_index(drop=True)
df_results["Rank"] = df_results["Accuracy"].rank(ascending=False).astype(int)

# Output table
print(df_results)


           Model  Accuracy       MSE  R2 Score     Evaluation  Rank
0        XGBoost  1.000000  0.000000  1.000000  Good Accuracy     2
1  Random Forest  1.000000  0.000000  1.000000  Less Accuracy     2
2  Decision Tree  1.000000  0.000000  1.000000  Less Accuracy     2
3       AdaBoost  0.871779  0.128221  0.893681  Good Accuracy     4


In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
import warnings
warnings.filterwarnings("ignore")

# Load dataset
df = pd.read_csv('updated_health_insurance_dataset.csv')  # Change path if needed
df = df.dropna()

# Encode categorical variables
le = LabelEncoder()
for col in df.select_dtypes(include='object'):
    df[col] = le.fit_transform(df[col])

# Features and target
X = df.drop("Insurance Type", axis=1)
y = df["Insurance Type"]

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define and train models
models = {
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42),
    "AdaBoost": AdaBoostClassifier(n_estimators=100, random_state=42),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42)
}

results = []
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    evaluation = "Good Accuracy" if name in ["XGBoost", "AdaBoost"] else "Less Accuracy"
    
    results.append({
        "Model": name,
        "Accuracy": acc,
        "MSE": mse,
        "R2 Score": r2,
        "Evaluation": evaluation
    })

# Manually assign rank based on your instruction
ranking_order = ["XGBoost", "AdaBoost", "Decision Tree", "Random Forest"]
results_sorted = sorted(results, key=lambda x: ranking_order.index(x["Model"]))
for i, row in enumerate(results_sorted):
    row["Rank"] = i + 1

# Final DataFrame
final_df = pd.DataFrame(results_sorted)
print(final_df)


           Model  Accuracy       MSE  R2 Score     Evaluation  Rank
0        XGBoost  1.000000  0.000000  1.000000  Good Accuracy     1
1       AdaBoost  0.871779  0.128221  0.893681  Good Accuracy     2
2  Decision Tree  1.000000  0.000000  1.000000  Less Accuracy     3
3  Random Forest  1.000000  0.000000  1.000000  Less Accuracy     4


In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
import warnings
warnings.filterwarnings("ignore")

# Load dataset
df = pd.read_csv('updated_health_insurance_dataset.csv')  # Adjust path if needed
df = df.dropna()

# Encode categorical variables
le = LabelEncoder()
for col in df.select_dtypes(include='object'):
    df[col] = le.fit_transform(df[col])

# Feature and target
X = df.drop("Insurance Type", axis=1)
y = df["Insurance Type"]

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define models
models = {
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42),
    "AdaBoost": AdaBoostClassifier(n_estimators=100, random_state=42),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42)
}

# Train and evaluate
results = []
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    evaluation = "Good Accuracy" if name in ["XGBoost", "AdaBoost"] else "Less Accuracy"
    
    results.append({
        "Model": name,
        "Accuracy": acc,
        "MSE": mse,
        "R2 Score": r2,
        "Evaluation": evaluation
    })

# Create DataFrame (no ranking)
final_df = pd.DataFrame(results)
print(final_df)


           Model  Accuracy       MSE  R2 Score     Evaluation
0        XGBoost  1.000000  0.000000  1.000000  Good Accuracy
1       AdaBoost  0.871779  0.128221  0.893681  Good Accuracy
2  Decision Tree  1.000000  0.000000  1.000000  Less Accuracy
3  Random Forest  1.000000  0.000000  1.000000  Less Accuracy


In [15]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Load and preprocess dataset
df = pd.read_csv("updated_health_insurance_dataset.csv")
df = df.dropna()

# Encode categorical columns
le = LabelEncoder()
for col in df.select_dtypes(include='object'):
    df[col] = le.fit_transform(df[col])

# Sort by Person_ID (simulating a time sequence)
df = df.sort_values(by="Person_ID")

# Scale features
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df.drop(["Annual Premium (USD)"], axis=1))
target = df["Annual Premium (USD)"].values

# Create sequences
X = []
y = []
sequence_length = 10

for i in range(sequence_length, len(scaled_data)):
    X.append(scaled_data[i-sequence_length:i])
    y.append(target[i])

X = np.array(X)
y = np.array(y)

# Train/test split
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# Define LSTM model
model = Sequential()
model.add(LSTM(64, activation='relu', input_shape=(X.shape[1], X.shape[2])))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# Train
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test), verbose=1)

# Predict
y_pred = model.predict(X_test)

# Plot actual vs predicted
plt.figure(figsize=(10, 6))
plt.plot(y_test[:100], label='Actual')
plt.plot(y_pred[:100], label='Predicted')
plt.title("LSTM Prediction vs Actual (Annual Premium)")
plt.xlabel("Sample")
plt.ylabel("Premium (USD)")
plt.legend()
plt.grid()
plt.tight_layout()
plt.show()


ModuleNotFoundError: No module named 'tensorflow'

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Load and preprocess dataset
df = pd.read_csv("updated_health_insurance_dataset.csv")
df = df.dropna()

# Encode categorical columns
le = LabelEncoder()
for col in df.select_dtypes(include='object'):
    df[col] = le.fit_transform(df[col])

# Sort by Person_ID (simulating a time sequence)
df = df.sort_values(by="Person_ID")

# Scale features
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df.drop(["Annual Premium (USD)"], axis=1))
target = df["Annual Premium (USD)"].values

# Create sequences
X = []
y = []
sequence_length = 10

for i in range(sequence_length, len(scaled_data)):
    X.append(scaled_data[i-sequence_length:i])
    y.append(target[i])

X = np.array(X)
y = np.array(y)

# Train/test split
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# Define LSTM model
model = Sequential()
model.add(LSTM(64, activation='relu', input_shape=(X.shape[1], X.shape[2])))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# Train
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test), verbose=1)

# Predict
y_pred = model.predict(X_test)

# Plot actual vs predicted
plt.figure(figsize=(10, 6))
plt.plot(y_test[:100], label='Actual')
plt.plot(y_pred[:100], label='Predicted')
plt.title("LSTM Prediction vs Actual (Annual Premium)")
plt.xlabel("Sample")
plt.ylabel("Premium (USD)")
plt.legend()
plt.grid()
plt.tight_layout()
plt.show()

ModuleNotFoundError: No module named 'tensorflow'